<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/comment_bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install transformers
!pip install -U sentence-transformers
!pip install bertopic==0.14.1
!pip install bertopic[flair]
!pip install bertopic[gensim]
!pip install bertopic[spacy]
!pip install bertopic[use]
!pip install emoji
!pip install advertools
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-3.20.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.3
    Uninstalling protobuf-4.22.3:
      Successfully uninstalled protobuf-4.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensor

In [32]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel

import sklearn as sk
import os
import nltk
from nltk.data import find

import seaborn as sns
import matplotlib.pyplot as plt

import re
import string
import advertools as adv


In [33]:
df = pd.read_csv("comments.csv")
df

,Business Unit Name,EVP,Facility Type,District,Comment Language,Original Comment Text,English Comment,Compound Sentiment Score,Overall Sentiment,spl_char_chk,deEmojify_text,emoji_present_flag,all_punct_chk,english_comment_length
0,05R-Denver,"Griffith, Kelly",OF,60,en,I can step down and go back to being a pharmac...,I can step down and go back to being a pharmac...,-0.6908,Negative,False,I can step down and go back to being a pharmac...,False,False,277
1,05R-Denver,"Griffith, Kelly",OF,60,en,I have worked more hours than I would have lik...,I have worked more hours than I would have lik...,-0.4005,Negative,False,I have worked more hours than I would have lik...,False,False,278
2,05R-Denver,"Griffith, Kelly",OF,60,en,"Corporate SLT ""Thank You"" for implementing “Li...","Corporate SLT ""Thank You"" for implementing “Li...",-0.0258,Negative,False,"Corporate SLT ""Thank You"" for implementing “Li...",False,False,461
3,05R-Denver,"Griffith, Kelly",OF,60,en,We have made incredible strides in the Denver ...,We have made incredible strides in the Denver ...,0.0258,Positive,False,We have made incredible strides in the Denver ...,False,False,296
4,05R-Denver,"Griffith, Kelly",OF,60,en,"Work on benefits, seems poorer every year.","Work on benefits, seems poorer every year.",0.0258,Positive,False,"Work on benefits, seems poorer every year.",False,False,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58232,United,NaN,NaN,NaN,en,This is an amazing company to work with. In ou...,This is an amazing company to work with. In ou...,0.9947,Positive,False,This is an amazing company to work with. In ou...,False,False,1068
58233,United,NaN,NaN,NaN,en,"ever since I started working here, I have neve...","ever since I started working here, I have neve...",0.9949,Positive,False,"ever since I started working here, I have neve...",False,False,896
58234,United,NaN,NaN,NaN,en,Very good at working with request for days off...,Very good at working with request for days off...,0.9971,Positive,False,Very good at working with request for days off...,True,False,3213
58235,United,NaN,NaN,NaN,en,"I love working at my job it’s my happy place, ...","I love working at my job it’s my happy place, ...",0.9977,Positive,False,"I love working at my job it’s my happy place, ...",False,False,781


In [16]:
dir(BERTopic)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_reduce_topics',
 '_c_tf_idf',
 '_cluster_embeddings',
 '_create_topic_vectors',
 '_extract_embeddings',
 '_extract_topics',
 '_extract_words_per_topic',
 '_get_param_names',
 '_map_predictions',
 '_map_probabilities',
 '_plotly_topic_visualization',
 '_preprocess_text',
 '_reduce_dimensionality',
 '_reduce_to_n_topics',
 '_reduce_topics',
 '_select_embedding_model',
 '_update_topic_size',
 'find_topics',
 'fit',
 'fit_transform',
 'get_params',
 'get_topic',
 'get_topic_freq',
 'get_topic_info',
 'get_topics',
 'load',
 'reduce_topics',
 'save',
 'topics_over_time',
 'transform',
 'update_topics',
 'visualize_distr

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distilbert-base-nli-stsb-mean-tokens
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [26]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
#topic_model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', vectorizer_model=None,min_topic_size= 15, nr_topics="auto", language = 'english',calculate_probabilities = True)
topic_model=BERTopic()
comment_text = df['deEmojify_text'].values.tolist()
headline_topics, probs = topic_model.fit_transform(comment_text)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distilbert-base-nli-stsb-mean-tokens
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
2023-04-28 18:41:38,900 - BERTopic - Transformed documents to Embeddings
2023-04-28 18:43:02,366 - BERTopic - Reduced dimensionality with UMAP
2023-04-28 18:43:11,077 - BERTopic - Clustered UMAP embeddings with HDBSCAN


AttributeError: ignored

In [ ]:
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

In [ ]:
a_topic = freq.iloc[1]["Topic"] # Select the 1st topic
topic_model.get_topic(a_topic) # Show the words and their c-TF-IDF scores

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
similar_topics, similarity = topic_model.find_topics("pay", top_n = 5)
similar_topics 

In [ ]:
freq[freq["Topic"].isin(similar_topics)]

In [ ]:
# Get the clusterer model, the clusters' tree and the clusters (topics ids)
clusterer = topic_model.hdbscan_model
tree = clusterer.condensed_tree_
clusters = tree._select_clusters()

# Get the ids of the most relevant documents (exemplars) associated with the topic at index idx
c_exemplars = topic_model.get_most_relevant_documents(clusters[idx], tree)

In [ ]:
most_similar = similar_topics[0]
print("Most Similar Topic Info: \n{}".format(model.get_topic(most_similar)))
print("Similarity Score: {}".format(similarity[0]))